In [9]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras import models
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,Normalizer, MinMaxScaler
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import GridSearchCV
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

In [11]:
df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name

day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)


In [12]:
def split_df(df):
    d={}
    split_by_list=np.unique(df.id.values)
    for sku in split_by_list:
        d[sku]=df[df.id==sku]
    return d

df_dict=split_df(df);

In [13]:
y={}
for sku in list(df_dict.keys()):
    y[sku]=df_dict[sku]['Sales']

In [14]:
X={}
for sku in list(df_dict.keys()):
    X[sku]=df_dict[sku]
    X[sku]=X[sku].drop(['id','DayOfWeek','Customers','High_Var','Luxury','Sales'],axis=1)

In [15]:
for sku in list(df_dict.keys()):
    for obs in range(1,8):
        X[sku]["Sales_T"+str(obs)]=df_dict[sku]['Sales'].shift(obs)

In [22]:
for sku in list(df_dict.keys()):
    X[sku]["Mov_avg"]=df_dict[sku]['Sales'].rolling( window=7).mean().shift(1)

In [25]:
#Cut lagged vars NAs off top 
cut_lag=7;
for sku in list(df_dict.keys()):
    y[sku]=y[sku][cut_lag:]
    y[sku].reset_index(drop=True, inplace=True)
    
for sku in list(df_dict.keys()):
    X[sku]=X[sku][cut_lag:]
    X[sku].reset_index(drop=True, inplace=True)

In [24]:
X[1].head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7,Mov_avg
0,1,1,1,2013,1,0,0,0,0,0,1,0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,0.0,3788.000000
1,1,1,1,2013,1,0,0,0,0,0,0,1,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,4585.142857
2,1,1,1,2013,1,0,0,0,0,1,0,0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,4576.714286
3,1,1,1,2013,1,1,0,0,0,0,0,0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4657.428571
4,1,0,0,2013,1,0,0,1,0,0,0,0,4881.0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4713.857143


In [26]:
std={}
for sku in list(df_dict.keys()):
    std[sku] = preprocessing.StandardScaler().fit(X[sku])
    X[sku] = std[sku].transform(X[sku])


In [27]:
cv=5

RF=RandomForestRegressor( n_estimators=1000,max_depth=10,max_features=14);
# use a full grid over all parameters


rf_models=[];
for sku in list(df_dict.keys()):
    rf_models.append(cross_val_score(RF,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0) )
#


In [28]:

GB=ensemble.GradientBoostingRegressor(n_estimators= 1000,max_depth= 8, max_features=14)#,learning_rate=0.01,loss='ls');
gb_models=[];
for sku in list(df_dict.keys()):
    gb_models.append(cross_val_score(GB,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
#    


In [29]:
XG=xgb.XGBRegressor(n_estimators= 900,learning_rate =0.01, max_depth=6,gamma=2  )
xg_models=[]
for sku in list(df_dict.keys()):
    xg_models.append(cross_val_score(XG,X[sku],y[sku],cv=cv, scoring ='mean_absolute_error').mean().round(0)  )

In [30]:
def create_MLP():
    mlp=models.Sequential()
    mlp.add(Dense(20, input_dim=X[1].shape[1], activation='relu'))
    mlp.add(Dense(30, activation='relu'))
    #mlp.add(Dense(30, activation='relu'))
    mlp.add(Dense(10, activation='relu'))
    mlp.add(Dense(1))
    mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
    return mlp

MLP = KerasRegressor(build_fn=create_MLP,
                               epochs=400,
                               batch_size=20,
                               verbose=0)

In [31]:
mlp_models=[]
for sku in list(df_dict.keys()):
    mlp_models.append(cross_val_score(MLP,X[sku],y[sku],cv=cv).mean().round(0)  )

In [32]:
mae=-pd.DataFrame(np.column_stack((mlp_models,rf_models,gb_models,xg_models)) )
mae=mae.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG'})
best=mae.idxmin(axis=1)
best.astype('category')
mae["Best"]=best
mae.index = mae.index + 1
mae.loc['mean'] = mae.mean()
mae['range']=mae.max(axis=1)-mae.min(axis=1)
mae

,MLP,RF,GB,XG,Best,range
1,378.0,339.0,353.0,357.0,RF,39.0
2,392.0,380.0,399.0,388.0,RF,19.0
3,595.0,554.0,595.0,566.0,RF,41.0
4,675.0,684.0,741.0,709.0,MLP,66.0
5,363.0,346.0,358.0,362.0,RF,17.0
6,416.0,390.0,401.0,403.0,RF,26.0
7,690.0,689.0,708.0,709.0,RF,20.0
8,462.0,429.0,444.0,443.0,RF,33.0
9,523.0,531.0,534.0,531.0,MLP,11.0
10,394.0,364.0,380.0,377.0,RF,30.0


In [33]:
#Need lots of rank and counts of how many times models came in 1st  vs 2nd.
def rank_model(df,rank):
    coln = 'Best' + str(rank) 
    sortID = np.argpartition(df[['MLP','RF','GB','XG']].values,rank,axis=1)[:,rank]
    df[coln] = df.columns[sortID]
    

In [34]:
rank_model(mae,0)
rank_model(mae,1)
mae

,MLP,RF,GB,XG,Best,range,Best0,Best1
1,378.0,339.0,353.0,357.0,RF,39.0,RF,GB
2,392.0,380.0,399.0,388.0,RF,19.0,RF,XG
3,595.0,554.0,595.0,566.0,RF,41.0,RF,XG
4,675.0,684.0,741.0,709.0,MLP,66.0,MLP,RF
5,363.0,346.0,358.0,362.0,RF,17.0,RF,GB
6,416.0,390.0,401.0,403.0,RF,26.0,RF,GB
7,690.0,689.0,708.0,709.0,RF,20.0,RF,MLP
8,462.0,429.0,444.0,443.0,RF,33.0,RF,XG
9,523.0,531.0,534.0,531.0,MLP,11.0,MLP,RF
10,394.0,364.0,380.0,377.0,RF,30.0,RF,XG


In [35]:
def RF_final(X, y):
    tree=RF
    tree.fit(X,y)
    pred=tree.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'RF'})
    return pred

def MLP_final(X,y):
    model = MLP
    model.fit(X,y)
    pred=model.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'MLP'})
    return pred

def GB_final(X,y):
    model = GB
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'GB'})
    return pred

def XG_final(X,y):
    model=XG
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'XG'})
    return pred

In [36]:
#NOTE, because we are retraining models here, a bad initiatlization could lead to different results. 

def best_model(scores,X,y):
    d={}
    for sku in list(df_dict.keys()):
        
        if scores.loc[sku]["Best"]=="RF":     
            d[sku]=RF_final(X[sku],y[sku])
                    
        elif scores.loc[sku]["Best"]=="MLP":
            d[sku]=MLP_final(X[sku],y[sku])                 
            
        elif scores.loc[sku]["Best"]=="GB":
            d[sku]=GB_final(X[sku],y[sku])
            
        elif scores.loc[sku]["Best"]=="XG":
            d[sku]=XG_final(X[sku],y[sku])
      
    return d     

In [37]:
bm_dict=best_model(mae,X,y)

In [38]:
#MAE of Best Model forecasting over all 935 days.  NN do very poorly here!
mae2=[]
for sku in list(df_dict.keys()):
    mae2.append( mean_absolute_error(y[sku], bm_dict[sku])   )
mae2=pd.DataFrame(mae2)
mae2.index = mae2.index + 1
mae2.round(1)

,0
1,193.6
2,216.2
3,305.3
4,507.5
5,197.2
6,190.5
7,351.5
8,202.0
9,383.8
10,199.7


In [44]:
sku_pred=pd.concat([ bm_dict[1],bm_dict[2],bm_dict[3],bm_dict[4],bm_dict[5],bm_dict[6],bm_dict[7],bm_dict[8],bm_dict[9],bm_dict[10],
],axis=1) 
sku_pred['Total'] = sku_pred.sum(axis=1)
sku_pred.columns = ['1','2','3','4','5','6','7','8','9','10','Total']
sku_pred.to_csv('C:\\Users\\lengada1\\NCSU\\prediction_skus.csv')

#Each column in this dataframe is the result of 1 of the 4 models
sku_pred

,1,2,3,4,5,6,7,8,9,10,Total
0,4028.070155,3933.607419,5174.478232,7432.938477,3725.016630,5607.321375,6806.916205,4207.681667,4355.298828,4575.576329,49846.905318
1,3939.086922,4632.660739,5142.490788,7006.601074,3997.977265,5275.288788,6387.782391,3888.736040,3897.496094,4557.150310,48725.270411
2,3890.810664,3840.314749,5054.772569,7085.299316,3719.835842,5551.322346,6798.710602,4839.750751,4251.476562,4520.816120,49553.109522
3,4253.015695,3886.589696,5778.068833,7839.360352,4058.536214,5698.072570,7586.416943,3999.500579,5356.418945,4761.602488,53217.582313
4,4986.052622,2519.789920,4131.315951,9462.297852,1784.665517,3649.817545,4877.134636,2423.252425,5014.551270,4516.572428,43365.450165
5,0.000000,0.000000,0.000000,0.000406,0.000000,0.000000,0.000000,0.000000,-0.000775,0.000000,-0.000370
6,5788.076745,6594.611178,7325.500858,11641.523438,6729.381251,7283.053659,10691.241413,7753.878928,7258.882812,6703.765011,77769.915294
7,5224.419822,5915.020785,8149.809159,9384.062500,5651.638970,7021.318303,9248.276299,6348.275655,5728.949219,5373.716902,68045.487613
8,5365.775298,6173.493269,8314.124895,9079.247070,5719.910999,6961.307500,8071.511607,6023.864150,4909.776367,5261.958235,65880.969391
9,5152.497215,5147.503253,7629.552835,8997.029297,4868.429738,6226.241809,7895.137287,6425.635110,4890.357910,4969.136484,62201.520938
